In [2]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from datetime import datetime
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [3]:
dffilm = pd.read_csv(Path('./Resources/train.csv'))

In [4]:
dffilm.drop(['belongs_to_collection','budget','genres','homepage',
            'imdb_id','spoken_languages','overview','title','poster_path','production_companies',
            'production_countries','original_title','tagline','Keywords','cast','crew'], axis=1, inplace=True)
dffilm.head()

,id,original_language,popularity,release_date,runtime,status,revenue
0,391,it,3.800073,12/22/12,0.0,Released,10703234
1,592,ru,0.402368,12/4/03,0.0,Released,234748
2,925,es,1.926826,11/15/13,0.0,Released,850259
3,978,it,5.010563,10/27/11,0.0,Released,12935800
4,1256,it,1.623440,8/25/75,0.0,Released,39598448


In [5]:

dffilm.revenue

0        10703234
1          234748
2          850259
3        12935800
4        39598448
          ...    
2995    122700000
2996     71000000
2997       871279
2998        31000
2999        65335
Name: revenue, Length: 3000, dtype: int64

In [6]:


X_releasedate =dffilm
for m in range(0,len(dffilm.release_date)):
    film_date = dffilm.release_date[m]
    dt = datetime.strptime(film_date, '%m/%d/%y')
    #dffilm.release_date[m] = datetime_object.timetuple().tm_yday
    X_releasedate.release_date[m]= dt.month
    
for i in range(0,len(dffilm.original_language)):
    if dffilm.original_language[i]== 'en':
        X_releasedate.original_language[i] = 1
    else:
        X_releasedate.original_language[i] = 0
        

for j in range(0,len(dffilm.status)):
    if dffilm.status[j]== 'Released':
        X_releasedate.status[j] = 1
    else:
        X_releasedate.status[j] = 0        
        
X_releasedate =X_releasedate[X_releasedate['runtime'] !=0]
X_releasedate =X_releasedate[X_releasedate['revenue'] !=0]
        
X_releasedate

/var/folders/23/mc3swhfd7ts230fqc9_j4xqw0000gn/T/ipykernel_9622/821945639.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_releasedate.release_date[m]= dt.month
/var/folders/23/mc3swhfd7ts230fqc9_j4xqw0000gn/T/ipykernel_9622/821945639.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_releasedate.original_language[i] = 0
/var/folders/23/mc3swhfd7ts230fqc9_j4xqw0000gn/T/ipykernel_9622/821945639.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

,id,original_language,popularity,release_date,runtime,status,revenue
12,2578,1,0.209191,6,11.0,1,97
13,776,1,2.923870,4,40.0,1,7732325
14,1021,1,1.081517,2,40.0,1,87600000
15,225,1,0.410738,5,53.0,1,500
16,1008,1,0.007877,2,60.0,0,60
...,...,...,...,...,...,...,...
2995,524,1,11.147040,10,220.0,1,122700000
2996,1923,1,8.591238,6,248.0,1,71000000
2997,1212,1,4.558075,5,338.0,1,871279
2998,1336,0,0.292296,10,NaN,1,31000


In [7]:
X_releasedate['release_date']=X_releasedate.release_date.astype('int64')
X_releasedate['original_language']=X_releasedate.original_language.astype('int64')
X_releasedate['status']=X_releasedate.status.astype('int64')

In [8]:
X = X_releasedate.dropna().copy()
X.head()

,id,original_language,popularity,release_date,runtime,status,revenue
12,2578,1,0.209191,6,11.0,1,97
13,776,1,2.923870,4,40.0,1,7732325
14,1021,1,1.081517,2,40.0,1,87600000
15,225,1,0.410738,5,53.0,1,500
16,1008,1,0.007877,2,60.0,0,60


In [9]:

X=X_releasedate.dropna()

X


,id,original_language,popularity,release_date,runtime,status,revenue
12,2578,1,0.209191,6,11.0,1,97
13,776,1,2.923870,4,40.0,1,7732325
14,1021,1,1.081517,2,40.0,1,87600000
15,225,1,0.410738,5,53.0,1,500
16,1008,1,0.007877,2,60.0,0,60
...,...,...,...,...,...,...,...
2993,1915,1,5.374563,2,214.0,1,12923936
2994,1303,1,4.393576,11,219.0,1,3484331
2995,524,1,11.147040,10,220.0,1,122700000
2996,1923,1,8.591238,6,248.0,1,71000000


In [10]:
y = X["revenue"].ravel()
y[:5]

array([      97,  7732325, 87600000,      500,       60])

In [11]:
X.drop("revenue", axis=1, inplace=True)
X

/Users/vikramjindal/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,id,original_language,popularity,release_date,runtime,status
12,2578,1,0.209191,6,11.0,1
13,776,1,2.923870,4,40.0,1
14,1021,1,1.081517,2,40.0,1
15,225,1,0.410738,5,53.0,1
16,1008,1,0.007877,2,60.0,0
...,...,...,...,...,...,...
2993,1915,1,5.374563,2,214.0,1
2994,1303,1,4.393576,11,219.0,1
2995,524,1,11.147040,10,220.0,1
2996,1923,1,8.591238,6,248.0,1


In [12]:
# Splitting into Train and Test sets
#X_train,X_test,y_train,y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train,X_test,y_train,y_test = train_test_split(X, y,random_state=78)

In [13]:
X_test
y_train
y_test

y_train



array([27642707, 35743308, 57284237, ...,  1192322,    10068,  2807390])

In [14]:
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [15]:
#Fitting the model
rf_model = rf_model.fit(X_train, y_train)

In [16]:
predictions = rf_model.predict(X_test)
print(predictions)

[   5151936  123726688  155760117   10000000   29400000   24600832
      22321   10676194    8403433     393970    1929168   50007546
   21567853   91038276  284139100   18552460    6700000  152427960
     124693  163403799     500000    2474000    1200398    1461989
   40084041    5790448   19179969    9109322    1246141    2405420
  190320568    2978994   25312387  266000000   62882090   77516304
   48428063   51881013   40084041   40940662    1829917   15992615
    3338228    1000000    9250000   18124262   10654385  850000000
   16337881   17425000      10068   14715067    5765562   66821036
   19738015  147332697    8900000   93282604    3566225    1524966
   11879046    3335984   14051384    5873871        641   17986781
      16136         23   10166502    6834525    4417667    1347988
     163000   21800302  173000000   48902953     100202     127251
   13000000        923  351692268    2500000    1547397     117629
   48720000    9600000  100914445   75715436   17300889  40006

In [19]:
#A_sorted = y_test.sort_values(["revenue"], ascending=True)
#y_test=A_sorted
# Calculating the confusion matrix
#cm = confusion_matrix(y_test, predictions)
#cm_df = pd.DataFrame(  cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)




In [20]:
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Accuracy Score : 0.004016064257028112
Classification Report
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         2
          11       0.00      0.00      0.00         0
          20       0.00      0.00      0.00         0
          23       0.00      0.00      0.00         0
          25       0.00      0.00      0.00         1
          46       0.00      0.00      0.00         0
          60       0.00      0.00      0.00         1
          85       0.00      0.00      0.00         0
         100       0.00      0.00      0.00         1
         135       0.00      0.00      0.00         0
         525       0.00      0.00      0.00         1
         641       0.00      0.00      0.00         0
         671       0.

/Users/vikramjindal/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vikramjindal/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vikramjindal/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users

In [21]:
#X_train.dtypes
#X_test
#y_train
#y_test
X.dtypes

id                     int64
original_language      int64
popularity           float64
release_date           int64
runtime              float64
status                 int64
dtype: object

In [21]:
dffilm['revenue'].describe()

count    3.000000e+03
mean     6.672585e+07
std      1.375323e+08
min      1.000000e+00
25%      2.379808e+06
50%      1.680707e+07
75%      6.891920e+07
max      1.519558e+09
Name: revenue, dtype: float64